# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import warnings

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
stop_words = stopwords.words("english")

warnings.filterwarnings("ignore")

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///messages_db.db')
df = pd.read_sql_table('tb_messages', engine)
X = df.message
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
targets = Y.columns.values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', string = text)
    
    # Todo: tokenize text
    tokens = nltk.tokenize.word_tokenize(text)
    
    # Todo: lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('count_vec', CountVectorizer(tokenizer = tokenize)),
    ('tfdf', TfidfTransformer()),
    ('random_forest', MultiOutputClassifier(RandomForestClassifier()))
]
)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
for i in range(36):
    print(f"{y_test.columns[i]} category:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

related category:
             precision    recall  f1-score   support

        0.0       0.33      0.22      0.26       656
        1.0       0.67      0.79      0.72      1366
        2.0       0.00      0.00      0.00        23

avg / total       0.55      0.60      0.57      2045

request category:
             precision    recall  f1-score   support

        0.0       0.65      0.88      0.75      1310
        1.0       0.42      0.16      0.23       735

avg / total       0.57      0.62      0.56      2045

offer category:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2041
        1.0       0.00      0.00      0.00         4

avg / total       1.00      1.00      1.00      2045

aid_related category:
             precision    recall  f1-score   support

        0.0       0.61      0.82      0.70      1220
        1.0       0.46      0.23      0.30       825

avg / total       0.55      0.58      0.54      2045

medical_help 

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'random_forest__estimator__max_depth': [10, 30, 50],
    'random_forest__estimator__min_samples_leaf': [1, 2, 4],
    'random_forest__estimator__min_samples_split': [2, 5, 10]
}
cv = GridSearchCV(pipeline, parameters, cv = 5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'random_forest__estimator__max_depth': [10, 30, 50], 'random_forest__estimator__min_samples_leaf': [1, 2, 4], 'random_forest__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:

y_pred = cv.best_estimator_.predict(X_test)
for i in range(36):
    print(f"{y_test.columns[i]} category:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

related category:
             precision    recall  f1-score   support

        0.0       1.00      0.00      0.00       656
        1.0       0.67      1.00      0.80      1366
        2.0       0.00      0.00      0.00        23

avg / total       0.77      0.67      0.54      2045

request category:
             precision    recall  f1-score   support

        0.0       0.64      1.00      0.78      1310
        1.0       1.00      0.00      0.01       735

avg / total       0.77      0.64      0.50      2045

offer category:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2041
        1.0       0.00      0.00      0.00         4

avg / total       1.00      1.00      1.00      2045

aid_related category:
             precision    recall  f1-score   support

        0.0       0.60      1.00      0.75      1220
        1.0       0.62      0.01      0.01       825

avg / total       0.61      0.60      0.45      2045

medical_help 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
pipeline = Pipeline([
    ('count_vec', CountVectorizer(tokenizer = tokenize)),
    ('tfdf', TfidfTransformer()),
    ('nb', MultiOutputClassifier(MultinomialNB()))
]
)
parameters = {
    'nb__estimator__alpha': [1.0, 1.5], 
}
cv = GridSearchCV(pipeline, parameters)
cv.fit(X, Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'nb__estimator__alpha': [1.0, 1.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
y_pred = cv.best_estimator_.predict(X_test)
for i in range(36):
    print(f"{y_test.columns[i]} category:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

related category:
             precision    recall  f1-score   support

        0.0       0.98      0.07      0.13       656
        1.0       0.68      1.00      0.81      1366
        2.0       0.00      0.00      0.00        23

avg / total       0.77      0.69      0.58      2045

request category:
             precision    recall  f1-score   support

        0.0       0.67      1.00      0.80      1310
        1.0       0.96      0.11      0.20       735

avg / total       0.77      0.68      0.58      2045

offer category:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2041
        1.0       0.00      0.00      0.00         4

avg / total       1.00      1.00      1.00      2045

aid_related category:
             precision    recall  f1-score   support

        0.0       0.65      0.99      0.79      1220
        1.0       0.96      0.21      0.34       825

avg / total       0.77      0.68      0.61      2045

medical_help 

### 9. Export your model as a pickle file

In [12]:
import pickle
with open('NaiveBayesClassifier.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.